<div><img src="https://www.fullstackpython.com/img/logos/scipy.png" width=300></div>

* soubor matematických funkcí a operací
* postavena na NumPy
* alternativa k MATLAB, Octave
* MATLAB 
  *  má prostředí, které usnadňuje práci
  *  licence
* Python 
  *  je více flexibilní
  *  zadarmo, open-source

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Speciální funkce
* různé matematické funkce a fyzikální vzorečky
* vstup a výstup funkcí jsou obvykle ndarray

In [ ]:
from scipy import special as sc

### př: gamma funkce

In [ ]:
x = np.linspace(0.1,5,1000)
y = sc.gamma(x)
plt.plot(x,y); # in jupyter it does not produce output

### př: beta funkce

In [ ]:
X, Y = np.meshgrid(x, x)
Z = sc.beta(X, Y)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z,)
ax.set_title('Beta function')
ax.set_xlabel('x')
ax.set_ylabel('y');

## Integrace

In [ ]:
from scipy import integrate

### Quad
* integrace na reálných číslech
* výstup je tuple (aproximace výsledku, chyba)
* využití lambda výrazů

In [ ]:
import math
integrate.quad(np.sin, 0, math.pi/2)

In [ ]:
integrate.quad(lambda x: np.exp(-x) , 0, np.inf)

In [ ]:
# ...Warning
integrate.quad(lambda x: x , 0, np.inf)

### dblquad, nquad
* dvojitá a n-tá integrace na reálných číslech
* meze integrace mohou být závislé na integrujicí proměnné
* dblquad a nquad pro n = 2 mají "prohozené" meze

In [ ]:
func1 = lambda x,z: x*x*z
integrate.dblquad(func1, 0, 1, 0, 5)  # boundaries for x are 0, 5

In [ ]:
func2 = lambda x,z: x*z
integrate.dblquad(func2, 0, 1, lambda z: 1+z, lambda z: 5-z)

In [ ]:
integrate.nquad(func1, [[0,5], [0, 1]])   # reverse order than dblquad

In [ ]:
integrate.nquad(func2, [lambda z: [1+z, 5-z], [0,1]])

### Aproximace integrace

* libovolné dělení intervalu
 * defaultně $dx = 1$
* trapz: lineární interpolace
* simps: průběh funkce mezi třemi body aproximuje parabolou

In [ ]:
aux = np.linspace(0, math.pi/2, 1000)
x = np.sin(aux)
integrate.trapz(x)  # ...default dx = 1.0

In [ ]:
aux = np.linspace(0, math.pi/2, 1000)
x = np.sin(aux)
integrate.trapz(x, aux)

In [ ]:
integrate.simps(x, aux)

In [ ]:
1.0 - integrate.simps(x, aux) < 1.0 - integrate.trapz(x, aux)

## Řešení ODR
př: Bratův problém:
* okrajová úloha na reálných číslech

$y'' + e^y = 0$

$y(0) = 0,\; y(1) = 0$

* potřeba transformace na systém ODR prvního řádu

$ y_0' = y_1$

$ y_1' = -e^{y_0}$

In [ ]:
func = lambda x,y: np.vstack((y[1], -np.exp(y[0])))

definujeme mřížku a okrajové podmínky:

In [ ]:
mesh = np.linspace(0, 1, 100)
bc = lambda x,y: np.array([x[0], y[0]])

připravíme pole pro výsledky:

In [ ]:
y = np.zeros((2, mesh.size))   # length of y in func is 2 

řešení:

In [ ]:
res = integrate.solve_bvp(func, bc, mesh, y)

In [ ]:
plt.plot(mesh, res.sol(mesh)[0]);

* Pro počáteční podmínky máme funkci *solve_ivp*

## Interpolace

### interp1d
* interpolace v 1d
* lineární interpolace
* instanci této třídy lze použít jako funkci ( \_\_call__ )

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
n = 10
x = np.linspace(0, 2*math.pi, n)
x_new = np.linspace(0, 2*math.pi, 10*n)
y = np.sin(x)
f1 = interp1d(x, y)
f2 = interp1d(x, y, kind='quadratic')
plt.plot(x, y, 'o', x_new, f1(x_new), x_new, f2(x_new), '--')
plt.legend(['data', 'linear', 'quadratic']);

* hodnota nejbližšího, předešlého a dalšího bodu 

In [ ]:
f1 = interp1d(x, y, kind='nearest')
f2 = interp1d(x, y, kind='previous')
f3 = interp1d(x, y, kind='next')
plt.plot(x, y, 'o', x_new, f1(x_new), x_new, f2(x_new), '--', x_new, f3(x_new), ':')
plt.legend(['data', 'nearest', 'previous', 'next']);

### griddata
* vícerozměrná interpolace
#### př:
* chceme interpolovat funkci $f:\mathbb{R}^2\longrightarrow\mathbb{R}$
* známe pouze hodnoty v bodech (x[i], y[i])
* body nevytvářejí pravidelnou mřížku

In [ ]:
from scipy.interpolate import griddata
x, y = np.mgrid[0:1:100j, 0:1:100j]
points = np.random.rand(1000, 2)           # decrease it to 100
def func(x, y):
    xs, ys = 0.5, 0.5
    R = 1
    return (xs-x)*(xs-x) + (ys-y)*(ys-y) - R*R
values = func(points[:,0], points[:,1])
f1 = griddata(points, values, (x, y))
plt.plot(points[:,0], points[:,1], 'k+')
plt.imshow(f1, extent=(0,1,0,1))
plt.colorbar();

## Statistika

### Spojitá rozdělení
* náhodná veličina s normálním rozdělením

In [ ]:
from scipy.stats import norm
from scipy.stats import gamma

In [ ]:
rvn = norm(loc=5, scale=3)
rvg = gamma(loc=5, scale=3, a=2)

* Kumulativní distribuční funkce

In [ ]:
x = np.linspace(0, 10, 1000)
plt.plot(x, rvn.cdf(x), x, rvg.cdf(x))
plt.legend(['norm', 'gamma']);

In [ ]:
rvn.cdf([0,5,10])

* hustota pravděpodobnosti

In [ ]:
plt.plot(x, rvn.pdf(x), x, rvg.pdf(x))
plt.legend(['norm', 'gamma']);

In [ ]:
rvn.pdf([0,5,10])

* kvantilová funkce

In [ ]:
plt.plot(x, rvn.ppf(x), x, rvg.ppf(x))
plt.legend(['norm', 'gamma']);

In [ ]:
rvn.ppf([0, 0.5, 1])

* střední hodnota
 * pro normální rozdělení odpovídá loc

In [ ]:
print('norm:  {}\ngamma: {}\n'.format(rvn.mean(), rvg.mean()))

* rozptyl
 * pro normální rozdělení odpovídá scale

In [ ]:
print('norm:  {}\ngamma: {}\n'.format(rvn.var(), rvg.var()))

* stats

In [ ]:
print('norm:  {}\ngamma: {}\n'.format(rvn.stats(), rvg.stats()))

pokud chceme získat zpět standardní rozdělení (loc = 0, scale = 1), tak transformujeme hustotu pravděpodobnosti následujícím způsobem

$f^*_{std}(x) = f(scale\cdot x + loc)$

a škálujeme tak, aby $A\int_{\mathbb{R}}f^*_{std}=1$

In [ ]:
x2 = np.linspace(0, 10, 100)
plt.plot(x2, gamma(loc=0, scale=1, a=2).pdf(x2), 'k+', x, rvg.pdf((3*x+5)),  x, 3*rvg.pdf((3*x+5)),  )
plt.legend(['std', 'trans', 'trans + scale']);

* standardní odchylka

In [ ]:
print('norm:  {}\ngamma: {}\n'.format(rvn.std(), rvg.std()))

* obecné momenty

In [ ]:
print('norm:  {}\ngamma: {}\n'.format(rvn.moment(1), rvg.moment(1)))

* generování sekvence náhodných čísel dle určitého rozdělení
 * používá numpy.random

nepreproduktibilní:

In [ ]:
rvn.rvs(5)

reproduktibilní:
 * RandomState

In [ ]:
rvn.rvs(5, 12345)

##### pozn:
* dle dokumentace bychom měli spíš používat BitGenerators a Generators, jak jsme si říkali v NumPy
* Proč?
 * RandomState je pro Legacy Generators

### Diskrétní rozdělení
* stejné metody, akorát pdf -> pmf
* scale neexistuje

In [ ]:
from scipy.stats import binom

In [ ]:
rvb = binom(n=15, p=0.5, loc=5)

In [ ]:
rvb.stats()

* Kumulační distribuční funkce

In [ ]:
x = np.arange(0, 21)
x_new = np.linspace(0, 20, 1000)
F = interp1d(x, rvb.cdf(x), kind='previous')
plt.plot(x_new, F(x_new));

In [ ]:
rvb.cdf([0, 1, 2, 3, 4, 5, 6])

* hustota pravděpodobnosti

In [ ]:
x = np.arange(0, 21)
plt.plot(x, rvb.pmf(x), 'o');

* kvantilová funkce
 * cdf je stupňovitá funkce

In [ ]:
x = np.arange(5, 21)
rvb.ppf(rvb.cdf(x))

In [ ]:
rvb.ppf(rvb.cdf(x)+1e-10)

### Tvoření vlastního rozdělení

#### spojité
 * tzn: subclassing rv_continuous

In [ ]:
from scipy import stats

In [ ]:
class MineDis(stats.rv_continuous):
    def _pdf(self, x):
        return np.where((-1 < x) & (x < 1), 0.5, 0)

In [ ]:
rvd = MineDis()
rvd.cdf(np.arange(-3, 3, 0.5))

In [ ]:
rvd.pdf(np.arange(-3, 3, 0.5))

In [ ]:
integrate.quad(rvd.pdf, -1, 1)